In [5]:
import nbimporter
from pathlib import Path
import pandas as pd
from catboost import CatBoostClassifier, Pool

from Preprop import * # load the data_preprocess function
from Feature_En import * # load the feature_generating function

In [6]:
d_path = 'datas'
params = {'depth': 5.0, 
          'l2_leaf_reg': 7.910707579888384, 
          'learning_rate': 0.329353775876958, 
          'loss_function': 0, 
          'max_bin': 33.0, 
          'random_strength': 0.13694748986533703
         }

In [7]:
def evaluate(model, X_test, y_test, model_name, use_proba=True):
    if use_proba == True:
        y_pred = model.predict_proba(X_test)[:, 1]
    elif use_proba == False:
        y_pred =  model.predict(X_test)
    ndcg = metrics.ndcg_score(y_test.values.reshape(1, -1), y_pred.reshape(1, -1), k=20)
    print(f'NDCG@{20} на тестовых данных c использованием {model_name}: {ndcg}')

In [8]:
df = data_preprocess(d_path, use_test=True)

D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x1 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x4 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [9]:
df = generate_features(df)

Feature_En.ipynb:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "            dataframes_profile.loc[:, item + '_' + featur + '_std'] = dataset.groupby(item)[featur].transform('std')\n",
Feature_En.ipynb:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        for featur in category_vars:\n",
Feature_En.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

MemoryError: Unable to allocate 2.96 GiB for an array with shape (40, 9934208) and data type float64

In [ ]:
df = df.loc[:, df.columns[3:]]
train_ind = df['target'][pd.isnull(df['target'])].index
test_ind = df['target'][pd.isnull(df['target'])].index

In [ ]:
X_train = df.drop('target', axis=1).loc[train_ind]
y_train = df['target'][train_ind]
X_test = df.drop('target', axis=1).loc[test_ind].reset_index(drop=True)
y_test = df['target'][test_ind]

In [ ]:
model = CatBoostClassifier(n_estimators=50, random_state=0, **params)
model.fit(X_train, y_train)

evaluate(model, X_test, y_test, 'CatBoostClassifier', 20, use_proba=False)
evaluate(model, X_test, y_test, 'CatBoostClassifier', 20, use_proba=True)